<a href="https://colab.research.google.com/github/emmanuelvaie/google_colab/blob/main/load_erg_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# piece of code that gets a token
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate the user
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# routine that gets a dataframe from the json file
import pandas as pd
def get_dataframe(id, file_name) :
  f_downloaded = drive.CreateFile({'id':id})
  f_downloaded.GetContentFile(file_name)
  with open(file_name, "r") as fid: 
     z = json.load(fid)
  # get results.participants
  df = pd.read_json(json.dumps(z['results']['participants']))
  return df


In [ ]:
import json
# get all the dataframes and concatenate the dataframes
file_list = drive.ListFile({'q': "trashed=false and title contains 'Test' and title contains '.json' and starred = true"}).GetList()
d_files = {}
i = 0
for file1 in file_list:
  d_files[file1['title']] = file1['id']
  print('Get file title: %s, id: %s' % (file1['title'], file1['id']))
  
  if i==0:
    df = get_dataframe(id= file1['id'], file_name= file1['title'])
  else:
    df_p = df
    df = get_dataframe(id= file1['id'], file_name= file1['title'])
    df = df_p.append(df,ignore_index=True)

  i = i+1
  

Get file title: Test ergo 28 Nov 10h30.json, id: 1J1Nnm-ocAysSZkY2n3HWwTvE1qk4CvAK
Get file title: Test ergo 28 Nov 10h15.json, id: 1sfCDupg1-Z5JbVNCZB0TRclyadJAqIR2
Get file title: Test ergo 28 Nov 10h.json, id: 1ijCEWswu-SW6ctXPJlQuebbZ2hdQfZ4n
Get file title: Test ergo 10h45.json, id: 1-STE_hriGgMqHFM0_h7dnxvM37Lvipr9
Get file title: Test ergo 10h30.json, id: 1U4-nTPoZ5gI9j4M1yFyiwrqT6gtVR4mE
Get file title: Test ergo 10h15.json, id: 13h_O8ocZCoWtt7lQuZnepXqVCmLu0rWd
Get file title: Test ergo 10h(0001).json, id: 1tBFZqQbVwCAhuek50KSm7d1E8d6G8Z-O


In [ ]:
df = df.drop(columns=['affiliation','splits','class','id','machine_type','serial_number'])

In [ ]:
# keep only rowers present during the race
df = df[df['score']!=0]

In [ ]:
df.

In [ ]:
# parse the time string
from datetime import datetime
df['duration'] = df['time'].apply(lambda x : datetime.strptime(x, "%M:%S.%f") )
df['formated-time'] = df['time'].apply(lambda x : datetime.strptime(x, "%M:%S.%f").strftime('%H:%M:%S.%f') )
df['formated-avg_pace'] = df['avg_pace'].apply(lambda x : datetime.strptime(x, "%M:%S.%f").strftime('%H:%M:%S.%f') )

# replace . by ,
df['formated-time'] = df['formated-time'].apply(lambda x : x.replace('.',','))
df['formated-avg_pace'] = df['formated-avg_pace'].apply(lambda x : x.replace('.',','))

In [ ]:
# sort the race
df.sort_values(by=['duration'], inplace=True)

In [ ]:
df.head()

,avg_pace,lane,logged_time,participant,place,score,spm,time,calories,duration,formated-time,formated-avg_pace
25,1:34.9,1,2021-11-21 10:34:00,CADDET,1,1000,36,3:09.8,89.0,1900-01-01 00:03:09.800,"00:03:09,800000","00:01:34,900000"
26,1:40.1,3,2021-11-21 10:34:00,VAIE,2,1000,35,3:20.3,NaN,1900-01-01 00:03:20.300,"00:03:20,300000","00:01:40,100000"
27,1:40.5,2,2021-11-21 10:34:00,VALLOIS,3,1000,32,3:21.1,82.0,1900-01-01 00:03:21.100,"00:03:21,100000","00:01:40,500000"
28,1:41.8,5,2021-11-21 10:34:00,LAWS,4,1000,33,3:23.8,80.0,1900-01-01 00:03:23.800,"00:03:23,800000","00:01:41,800000"
0,1:43.4,1,2021-11-28 10:52:00,CHARRIOU,1,1000,36,3:26.9,NaN,1900-01-01 00:03:26.900,"00:03:26,900000","00:01:43,400000"


In [ ]:
df = df[['avg_pace','calories', 'logged_time','participant','spm','time','formated-time','formated-avg_pace']]

In [ ]:
# Get the Google sheet
import gspread
from oauth2client.client import GoogleCredentials

import pandas as pd
worksheet_name ='https://docs.google.com/spreadsheets/d/1mseQhO9Bj0YeUqsjODj8Bu9BpF_E_bJAgYMB6jv0W4Q/edit#gid=2087892467'

gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open_by_url(worksheet_name).worksheet('Data')
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
df_info = pd.DataFrame.from_records(rows,columns=rows[0])
df_info = df_info.drop(0).reindex()
df_info['NOM'] = df_info['NOM'].apply(lambda x : x.strip())

In [ ]:
df = df.rename(columns={'participant':'NOM'})
df['NOM'] = df['NOM'].apply(lambda x : x.strip())

In [ ]:
# attention a la jointure, c'est une jointure sur la colonne NOM de df versus l'index de df_info
df_info_indexed = df_info.set_index('NOM')
df_joined = df.join(df_info_indexed,on='NOM',how='inner')

In [ ]:
df_joined['Type_test'] = 'ERGO 1000m'

In [ ]:
df_joined['Date test'] = df_joined['logged_time'].apply(lambda x : x.strftime('%d/%m/%Y'))

In [ ]:
df_joined = df_joined[['ID','Type_test','Date test','formated-time','AGE','SEXE', 'NOM','CAT']]
df_joined = df_joined.rename(columns={'Type_test':'TYPE_TEST','Date test':'DATE','formated-time':'TEMPS'})

In [ ]:
df_joined.head()

,ID,TYPE_TEST,DATE,TEMPS,AGE,SEXE,NOM,CAT
25,557989,ERGO 1000m,21/11/2021,"00:03:09,800000",24,H,CADDET,S
26,348574,ERGO 1000m,21/11/2021,"00:03:20,300000",50,H,VAIE,D
27,282175,ERGO 1000m,21/11/2021,"00:03:21,100000",32,H,VALLOIS,A
28,295016,ERGO 1000m,21/11/2021,"00:03:23,800000",46,H,LAWS,C
0,397922,ERGO 1000m,28/11/2021,"00:03:26,900000",26,H,CHARRIOU,S


In [ ]:
# get former results
worksheet = gc.open_by_url(worksheet_name).worksheet('Tests')
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
df_old = pd.DataFrame.from_records(rows,columns=rows[0])
df_old = df_old.drop(0).reindex()
df_old.head()

,ID,TYPE_TEST,DATE,TEMPS,AGE,SEXE,NOM,CAT
1,7313,ERGO 1000m,07/12/2019,00:03:19,46,H,RAIN,C
2,38647,ERGO 500m,24/03/2021,00:01:52,63,H,ACHACHE,F
3,370251,ERGO 1000m,07/12/2019,00:03:23,66,H,ALLOWAY,G
4,370251,ERGO 500m,24/03/2021,00:01:35,66,H,ALLOWAY,G
5,282175,ERGO 1000m,07/12/2019,00:03:19,32,H,VALLOIS,A


In [ ]:
df_all = df_old.append(df_joined)
df_all.head()

,ID,TYPE_TEST,DATE,TEMPS,AGE,SEXE,NOM,CAT
1,7313,ERGO 1000m,07/12/2019,00:03:19,46,H,RAIN,C
2,38647,ERGO 500m,24/03/2021,00:01:52,63,H,ACHACHE,F
3,370251,ERGO 1000m,07/12/2019,00:03:23,66,H,ALLOWAY,G
4,370251,ERGO 500m,24/03/2021,00:01:35,66,H,ALLOWAY,G
5,282175,ERGO 1000m,07/12/2019,00:03:19,32,H,VALLOIS,A


In [ ]:
# save the results in a Google sheet
tab = 'Results-Masters-Nov-2021'
try :
  results = worksheet.spreadsheet.worksheet(tab)
  results.clear()
except:
  results = worksheet.spreadsheet.add_worksheet(title=tab, rows="350", cols="20")

from gspread_dataframe import set_with_dataframe
set_with_dataframe(results, df)

In [ ]:
# save the results in a Google sheet
tab = 'All-results'
try :
  results = worksheet.spreadsheet.worksheet(tab)
  results.clear()
except:
  results = worksheet.spreadsheet.add_worksheet(title=tab, rows="350", cols="20")

from gspread_dataframe import set_with_dataframe
set_with_dataframe(results, df_all)